### Simplified Hilbert Space

In [ ]:
from qutip import *
import numpy as np
import scipy.optimize as sciop
import matplotlib.pyplot as plt
import cython
import skopt

# define idealised basis
basis_labels = {
    'cav_bin': 0,
    'spont_bin': 1,
    'g':2,
    'u':3,
    'x':4
}
N=len(basis_labels)

g_ket = basis(N, basis_labels['g'])
u_ket = basis(N, basis_labels['u'])
x_ket = basis(N, basis_labels['x'])

# define basic operators
gx_swap = basis(N, basis_labels['g']) * basis(N, basis_labels['x']).dag()
ux_swap = basis(N, basis_labels['u']) * basis(N, basis_labels['x']).dag()

g_id = fock_dm(N, basis_labels['g'])
u_id = fock_dm(N, basis_labels['u'])
x_id = fock_dm(N, basis_labels['x'])
cav_id = fock_dm(N, basis_labels['cav_bin'])
spont_id = fock_dm(N, basis_labels['spont_bin'])

cav_decay = basis(N, basis_labels['cav_bin']) * basis(N, basis_labels['g']).dag()
spont_decay = basis(N, basis_labels['spont_bin']) * basis(N, basis_labels['x']).dag()

### Parameters

In [ ]:
delta_stirap = 1
delta_C = delta_stirap
delta_L = delta_stirap

gamma = 1
kappa = gamma*1

coops = [2,5,10,20,50,100,125,150,200, 300]
omg_guesses = [238,238,238,234,230,226,225,223,220,219]


RUNTIME = 1/gamma
STEPS_PER_SEC = 1000
NUM_STEPS = round(STEPS_PER_SEC*RUNTIME + 1)
if type(RUNTIME) == float:
    if RUNTIME < 1:
        lin_factor = 1/RUNTIME
        RUNTIME = lin_factor*RUNTIME
    else:
        lin_factor = 10
        RUNTIME = np.ceil(RUNTIME*10)
t = np.linspace(0, int(RUNTIME), NUM_STEPS)
t /= lin_factor

print(t)
# define pulse shape
wSTIRAP = np.pi/RUNTIME
H_args = {
    'wSTIRAP': wSTIRAP
}
flat_pulse = '1'
sin_pulse = 'np.sin(wSTIRAP*t)**2'

c_ops = [np.sqrt(gamma)*spont_decay, np.sqrt(kappa)*cav_decay]

### Efficiency Functions

In [ ]:
def find_efficiency(omega, delta, cooperativity, psi0=u_ket, pulse_shape=sin_pulse):

    g0 = (kappa*gamma*cooperativity)**0.5

    theta = np.arctan(omega/2*g0)

    H0 = delta*g_id + delta*u_id - g0*(gx_swap + gx_swap.dag())
    H1 = -omega/2*(ux_swap + ux_swap.dag())
    H=[H0, [H1, pulse_shape]]

    result = mesolve(H, psi0, t, c_ops, [], args=H_args)

    photon_emission_prob = expect(cav_id, result.states[-1])
    spont_decay_prob = expect(spont_id, result.states[-1])
    # print(f'norm: {photon_emission_prob + spont_decay_prob}')

    return photon_emission_prob/(photon_emission_prob + spont_decay_prob)


### Optimisation

In [ ]:
max_delta = 50

def optimise_peak_rabi(cooperativity, omega_guess, delta_guess):

    def find_neg_efficiency(omega_delta, psi0=u_ket, pulse_shape=sin_pulse):

        omg = omega_delta[0]
        delta = omega_delta[1]

        g0 = (kappa*gamma*cooperativity)**0.5

        H0 = delta*g_id + delta*u_id - g0*(gx_swap + gx_swap.dag())
        H1 = -omg/2*(ux_swap + ux_swap.dag())
        H=[H0, [H1, pulse_shape]]

        result = mesolve(H, psi0, t, c_ops, [], args=H_args)

        photon_emission_prob = expect(cav_id, result.states[-1])
        spont_decay_prob = expect(spont_id, result.states[-1])

        return -photon_emission_prob/(photon_emission_prob + spont_decay_prob)

    
    return skopt.gp_minimize(find_neg_efficiency, [[omega_guess/5, 5*omega_guess], [0, max_delta]],x0=[omega_guess, delta_guess], n_calls=50)

In [ ]:
opt_results = []
bench1 = []
for C in coops:
    opt_results.append(optimise_peak_rabi(C, omg_guesses[coops.index(C)], 0))
    bench1.append(2*C/(2*C+1))

### Data Extraction

In [ ]:
opt_omgs = []
opt_delts = []
opt_effs = []

i = 0
for result in opt_results:
    opt_omgs.append(result.x[0])
    opt_delts.append(result.x[1])
    opt_effs.append(-result.fun)
    if result.x[1] > max_delta - 1:
        print(f'Delta optimisation failed for C = {coops[i]}')
    i += 1

### Plots

In [ ]:
fig, axes = plt.subplots(1,1)
axes.plot(coops, opt_omgs)
axes.set_xlabel('C')
axes.set_ylabel('$\Omega_P$')

fig, axes = plt.subplots(1,1)
axes.plot(coops, opt_effs)
axes.set_xlabel('C')
axes.set_ylabel('Efficiency')